In [3]:
import os
from google import genai
from google.genai import types#type: ignore
from dotenv import load_dotenv#type: ignore
import PyPDF2#type: ignore
import speech_recognition as sr#type: ignore
import os
import sounddevice as sd#type: ignore
import numpy as np#type: ignore
from google import genai


In [4]:
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_API_KEY)

In [5]:
resume_analysis_results = {}
jd_analysis_results = {}

In [6]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    try:
        text = ""
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""

In [7]:
def analyze_job_description(jd_path):
    """Analyzes a job description PDF and extracts key features."""
    try:
        text = extract_text_from_pdf(jd_path)
        if not text:
            return ""

        prompt = f'''Analyze the following job description:
                    {text}

                    Provide a structured summary: 
                    - Job Title
                    - Key Responsibilities
                    - Essential Technical Skills
                    - Desired Soft Skills
                    - Relevant Experience
                    - Education Requirements
                    
                    Generate 6 technical and 4 non-technical interview questions.
                    Highlight red flags for candidate responses.
                '''

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt],
        )

        return response.text
    except Exception as e:
        print(f"An error occurred while processing {jd_path}: {e}")
        return ""

In [8]:
def analyze_resume(pdf_path, jd_analysis):
    """Analyzes a resume against the job description analysis."""
    try:
        resume_text = extract_text_from_pdf(pdf_path)
        if not resume_text:
            return ""

        prompt = f'''Compare the following resume with the given job description.

                    Job Description Analysis:
                    {jd_analysis}

                    Candidate Resume:
                    {resume_text}

                    Provide a structured evaluation:
                    - Technical Skills Score (Max: 60)
                    - Experience Score (Max: 20)
                    - Soft Skills Score (Max: 10)
                    - Education Score (Max: 10)
                    - Total Score (Sum of all categories)

                    Generate 6 technical and 4 non-technical interview questions.
                    Highlight red flags for candidate responses.
                '''

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[prompt],
        )

        return response.text
    except Exception as e:
        print(f"An error occurred while processing {pdf_path}: {e}")
        return ""


In [9]:
conversation_history = []
asked_questions = {}

def interview_process(command_text, candidate_name):
    """Handles the interview process by mapping candidate responses to relevant interview questions."""
    if candidate_name not in resume_analysis_results:
        print(f"No resume analysis found for {candidate_name}.")
        return

    jd_analysed = jd_analysis_results.get("JD")
    resume_analysed = resume_analysis_results[candidate_name]

    # Initialize conversation history and asked questions
    
    if candidate_name not in asked_questions:
        asked_questions[candidate_name] = set()

    # Store the candidate's response for continuity
    
    conversation_history.append({"Input By": "user", "Query": command_text}),"\n"


    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=f'''You are an HR interviewer. The job description is {jd_analysed}.
                    The resume report for {candidate_name} is {resume_analysed}.
                    
                    
                    
                    The following questions have already been asked: {asked_questions[candidate_name]}.
                    Respond to {command_text} everytime. {command_text} is the response from the candidate. So you should build the conversation with 
                    respect to it. 
                    Avoid repeating these unless the candidate explicitly asks.
                    Ask relevant questions from the {resume_analysed} to test the knowledge of the person.
                    
                    If the candidate says "Pardon?", "I didn't get the question","can you explain once more", "i didnt get you" or anything unclear, **simplify and rephrase the question instead of repeating**.

                    Keep the interview engaging, informative, and structured.
                    If the candidate shows uncertainty ("I don't know", "I'm not sure"), reassure them with "It's fine" and move on.
                    
                    Conduct the interview in a natural, conversational manner and ensure a meaningful flow.
                    At the end, summarize the candidate’s performance, mentioning any red flags and strengths.
                '''
    )

    # Track the question to prevent repetition
    asked_questions[candidate_name].add(response.text.strip())

    conversation_history.append({"Output By": "assistant", "Response": response.text})

    print("AI Interviewer: ",response.text)


In [10]:


def listen_to_command():
    """Uses speech recognition to capture user input with extended listening duration."""
    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.pause_threshold = 1.2
        r.adjust_for_ambient_noise(source, duration=1)  # Adjust to background noise
        print("Please Answer..")

        try:
            # Wait for up to 30 seconds for the candidate to start speaking
            # Allow up to 60 seconds to complete the response
            audio_text = r.listen(source, timeout=30, phrase_time_limit=60)  
            print("Processing your command...")

            command_text = r.recognize_google(audio_text)
            print(f"Text: {command_text}")
            return command_text

        except sr.UnknownValueError:
            print("Sorry, I didn't catch that. Could you repeat?")
        except sr.RequestError:
            print("Sorry, the speech recognition service is unavailable.")
        except sr.WaitTimeoutError:
            print("No response detected. Take your time, but please answer when ready.")

    return None


***This is the part where the job description and resumes are analysed, Rated and possible technical and non technical questions are framed. Also for each question a red flag response has been marked.***

In [11]:
def process():
    """Processes job descriptions and resumes, storing their analysis for later use."""
    global resume_analysis_results, jd_analysis_results

    pdf_paths = {
        "John Doe": r"E:\Farmwise\full-stack-developer-resume-example.pdf",
        "Pranav": r"E:\Farmwise\Pranav_DataScience_Resume.pdf",
    }
    jd_path = r"E:\Farmwise\Job Description.pdf"

    print("Analyzing Job Description...")
    jd_analysis_results["JD"] = analyze_job_description(jd_path)
    print(jd_analysis_results["JD"])
    print("-" * 20)

    for name, pdf_path in pdf_paths.items():
        print(f"Analyzing Resume for {name}...")
        resume_analysis_results[name] = analyze_resume(pdf_path, jd_analysis_results["JD"])
        print(resume_analysis_results[name])
        print("-" * 20)

if __name__ == "__main__":
    process()  



Analyzing Job Description...
Okay, let's analyze the FarmwiseAI Internship to FTE job description and generate relevant interview questions.

**Structured Summary:**

*   **Job Title:** Intern – Data Science | Mobile App Development | Full Stack Development (with potential for Full-Time Employment)
*   **Key Responsibilities:**

    *   **Data Science:** Developing and fine-tuning LLMs and Generative AI solutions, data processing, model training/evaluation, AI research and implementation.
    *   **Mobile App Development:** Developing cross-platform mobile apps using Flutter, implementing UI features, integrating with Firebase/REST APIs, debugging, and optimizing app performance.
    *   **Full Stack Development:** Building and maintaining web apps (Python/Django/Flask backend, React.js frontend), developing REST APIs, managing databases, deploying to cloud platforms, debugging, and optimizing.
*   **Essential Technical Skills:**

    *   **Data Science:** Python, Machine Learning, NLP

***This is the part where the actual interview takes place. Speech recogoniton listens to the input audio, transcribes it, sends it to the model where processing takes places and output are given accordingly***

In [14]:
def voicebot():
    """Runs the HR interview model in a loop."""

    
    print("Welcome to the HR Interview Model.")
    print("Please say your name.")

    candidate_name = listen_to_command()
    if not candidate_name:
        print("No valid name detected. Exiting.")
        return

    candidate_name = candidate_name.strip()
    if candidate_name not in resume_analysis_results:
        print(f"No resume found for {candidate_name}. Please check the name.")
        return

    print(f"Hello {candidate_name}, let's begin your interview.")

    while True:
        
        user_response = listen_to_command()
        if not user_response:
            continue

        if user_response.lower() in ["exit", "quit", "stop"]:
            print("Interview session ended.")
            break

        interview_process(user_response, candidate_name)
if __name__ == "__main__":
    
    voicebot()  # Start the voice-based interview

Welcome to the HR Interview Model.
Please say your name.
Please Answer..
Processing your command...
Text: Pranav
Hello Pranav, let's begin your interview.
Please Answer..
Processing your command...
Text: yes please
AI Interviewer:  Okay Pranav, thanks for joining me today. Let's dive right in. I see you've worked with BERT in your NLP Voice Assistant project. Can you elaborate on the specific challenges you faced when integrating BERT and how you addressed them? Did you fine-tune the model, and if so, what datasets did you use?

Please Answer..
Processing your command...
Text: so what is basically bidirectional Bidi
AI Interviewer:  Okay, so BERT is basically bidirectional. That's a good starting point, Pranav. To dig a little deeper, can you tell me about the *specific* advantages of BERT's bidirectionality compared to earlier, unidirectional models like word2vec or GloVe, particularly in the context of your NLP Voice Assistant project? What impact did that bidirectionality have on th

***Conversation History is printed here***

In [15]:
print("CONVERSATION HISTORY")
for entry in conversation_history:
    for key, value in entry.items():
        final_history = (f"{key}: {value}")
        print(final_history)
    

CONVERSATION HISTORY
Input By: user
Query: yes please
Output By: assistant
Response: Okay Pranav, thanks for joining me today. Let's dive right in. I see you've worked with BERT in your NLP Voice Assistant project. Can you elaborate on the specific challenges you faced when integrating BERT and how you addressed them? Did you fine-tune the model, and if so, what datasets did you use?

Input By: user
Query: so what is basically bidirectional Bidi
Output By: assistant
Response: Okay, so BERT is basically bidirectional. That's a good starting point, Pranav. To dig a little deeper, can you tell me about the *specific* advantages of BERT's bidirectionality compared to earlier, unidirectional models like word2vec or GloVe, particularly in the context of your NLP Voice Assistant project? What impact did that bidirectionality have on the performance or capabilities of the assistant?

Input By: user
Query: but understands the context for example let's say we have two sentences for him runs the 